In [1]:
import pandas as pd
import numpy as np
from glob import glob
import warnings
warnings.filterwarnings('ignore')
import io
from sklearn.metrics import mean_absolute_error

## 1. 기상청 단기예보+중기예보 가져와서 10일치 예보 만들기 

In [2]:
data_year_temperature = pd.read_csv('./data/청운효자동_3시간기온_201812_201812.csv')  # 3시간 기온데이타
data_year = data_year_temperature[[' format: day', 'hour', 'forecast']]   # 하나의 데이타프레임에 기온데이타 만들기 
data_year['Temperature'] = data_year_temperature[data_year_temperature.columns[-1]]  

In [3]:
year =data_year_temperature.columns[3][31:35]  # year는 2018
month = data_year_temperature.columns[3][35:37] #month는 12

date = f'{year}-{month}-' +data_year[' format: day'].astype(str).str.split(' ').str[-1] + ' ' + (data_year['hour'].astype(int)//100).astype(str) + ':00' # date에 년월일데이타 만들기 
date = pd.to_datetime(date) + pd.DateOffset(hours=9) 

data_fcst = pd.DataFrame(columns=['Forecast time', 'forecast', 'Temperature'                       
                                  ])   # DataFrame 만들어 
data_fcst['Forecast time'] = date # date 채우기  
data_fcst[['forecast', 'Temperature']] = data_year[['forecast', 'Temperature'                                                               
                                                                 ]]

# Forecast time에 forecast 시간 더해서 Forecast_time 만들기 
fcst_data = data_fcst.copy() 

forecastchange=[]
forecast = fcst_data['forecast'].tolist() 
forecastTime= fcst_data['Forecast time'].tolist()
for f,t in zip(forecast,forecastTime):
    forecastchange.append(t + pd.DateOffset(hours=f))     
fcst_data['Forecast_time'] = forecastchange 

In [4]:
fcst_data.isnull().sum()

Forecast time    0
forecast         0
Temperature      0
Forecast_time    0
dtype: int64

In [5]:
# Forecast_time이 2019년인것만 가져오고 duplicated drop
fcst_data = fcst_data[['Forecast_time','Temperature']]
fcst_data = fcst_data.drop_duplicates(subset=['Forecast_time'], keep='last').reset_index(drop=True)
fcst_data

,Forecast_time,Temperature
0,2018-12-01 15:00:00,11.0
1,2018-12-01 18:00:00,7.0
2,2018-12-01 21:00:00,7.0
3,2018-12-02 00:00:00,6.0
4,2018-12-02 03:00:00,4.0
...,...,...
263,2019-01-03 12:00:00,-1.0
264,2019-01-03 15:00:00,2.0
265,2019-01-03 18:00:00,-1.0
266,2019-01-03 21:00:00,-1.0


In [6]:
#날짜 DataFrame만들기 
date = pd.DataFrame(pd.date_range(start='2019-01-01', end='2019-01-04', freq='H', name='Forecast_time', closed='left'))

In [7]:
# fcst_data와 합쳐서 보간하기 
fcst_data_ = pd.merge(fcst_data,date, how='outer', on='Forecast_time')
fcst_data_ = fcst_data_.sort_values(by='Forecast_time').reset_index(drop=True)

In [8]:
fcst_data__ = fcst_data_[243:].reset_index(drop=True) # 1월만 자르기 

In [9]:
fcst_data_inter = fcst_data__.interpolate()

In [10]:
# 중기예보 
fcst_midterm = pd.read_csv('./data/FCT_MEDM_20211203165632.csv', encoding='CP949')
fcst_midterm = fcst_midterm[(fcst_midterm['도시']=='서울') & (fcst_midterm['발표시각'].str.contains('2018-12-31 18시'))] # 서울/오후6시예보만 가져와서 
#날짜 dataframe만들기 
date2 = pd.DataFrame( pd.date_range(start='2019-01-03', end='2019-01-11', freq='H',name='Forecast_time' , closed='left'))

In [11]:
# 중기예보의 최저기온,최고기온 리스트로 만들어 date2 'Temperature' 열 ## 오전6시에 최저기온, 오후3시에 최고기온으로 채우기 (바꿔가며해봄)
low = fcst_midterm['최저기온'].tolist()
high = fcst_midterm['최고기온'].tolist()
date2['Temperature']=np.NAN

# 최저기온 채우기 
for i,j in enumerate(low): 
    date2.loc[24*i+6,'Temperature'] = j 

#최고기온 채우기 
for i,j in enumerate(high):
    date2.loc[24*i+15,'Temperature'] = j 

date2 = date2.drop(date2.index[0:23],axis=0) # 1/3일은 단기예보와 겹치므로 drop 

In [12]:
date2

,Forecast_time,Temperature
23,2019-01-03 23:00:00,NaN
24,2019-01-04 00:00:00,NaN
25,2019-01-04 01:00:00,NaN
26,2019-01-04 02:00:00,NaN
27,2019-01-04 03:00:00,NaN
...,...,...
187,2019-01-10 19:00:00,NaN
188,2019-01-10 20:00:00,NaN
189,2019-01-10 21:00:00,NaN
190,2019-01-10 22:00:00,NaN


In [13]:
# 단기예보와 합쳐서 보간하기
temp_for10 = pd.concat([fcst_data_inter,date2],axis=0)

temp_for10['Temperature']= temp_for10['Temperature'].astype(float)
temp_for10_inter = temp_for10.interpolate()
temp_for10_inter = temp_for10_inter.drop_duplicates(subset=['Forecast_time'])

In [14]:
temp_for10_inter.to_csv('./make_data/0D_weather_10_float.csv', index=False) # 단기예보 3일치+중기예보 7일치 합쳐서 10일 예보만듬. 

##  2. 관측치 30년치에서 여러가지 평균내보고 10일 예보와 합쳐 관측치와 오차점수 보기 

In [15]:
obs_data_path = './data/temp_30years' # 30년치 1988부터 
obs_data_list = sorted(glob(obs_data_path+'/*.csv')) # 기상청 관측 (10년치)  

In [16]:
obs_data_list #2008년부터 2018년까지 (음력날짜반영위해 11년) 
# 30년, 1988년부터 2018년까지 

['./data/temp_30years\\OBS_ASOS_TIM_20211201155755.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201155924.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160002.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160050.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160116.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160157.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160232.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160252.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160312.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201160608.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211201162736.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211202151116.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211202152120.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211202152141.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211202152806.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211202153932.csv',
 './data/temp_30years\\OBS_ASOS_TIM_20211202154001.csv',
 './data/temp_30years\\OBS_ASOS

In [17]:
df_merge = pd.DataFrame() # 데이터 합치기 
for idx,filename in enumerate(obs_data_list):
    df = pd.read_csv(filename, encoding='CP949',usecols=['일시','기온(°C)'])
    df_merge = df_merge.append(df,ignore_index=True)

In [18]:
df_merge = df_merge.sort_values('일시').reset_index(drop=True) 

In [19]:
df_merge_date = pd.DataFrame(pd.date_range(start='1988-01-01', end='2019-01-01',freq = 'h',name='일시', closed='left')) #30년치 날짜 만들기  
df_merge_date['일시'] = df_merge_date['일시'].astype(str).str[:16]
print(df_merge_date.shape) 

(271752, 1)


In [20]:
df_date = pd.merge(df_merge_date, df_merge,how='left',on='일시')
df_date

,일시,기온(°C)
0,1988-01-01 00:00,-3.7
1,1988-01-01 01:00,NaN
2,1988-01-01 02:00,NaN
3,1988-01-01 03:00,-4.4
4,1988-01-01 04:00,NaN
...,...,...
271747,2018-12-31 19:00,-3.1
271748,2018-12-31 20:00,-3.7
271749,2018-12-31 21:00,-4.6
271750,2018-12-31 22:00,-5.4


In [21]:
train_temp = df_date[219168:].reset_index(drop=True)
train_temp.columns=['date','temp']
train_temp

,date,temp
0,2013-01-01 00:00,-8.3
1,2013-01-01 01:00,-8.5
2,2013-01-01 02:00,-8.4
3,2013-01-01 03:00,-8.1
4,2013-01-01 04:00,-8.2
...,...,...
52579,2018-12-31 19:00,-3.1
52580,2018-12-31 20:00,-3.7
52581,2018-12-31 21:00,-4.6
52582,2018-12-31 22:00,-5.4


In [22]:
train_temp.isnull().sum()

date    0
temp    6
dtype: int64

In [23]:
train_temp = train_temp.interpolate()

In [24]:
train_temp.to_csv('./make_data/0D_train_temp_서울.csv',index=False) # 훈련데이터:사용 

In [25]:
# 온도데이터와 날짜데이터 병합 하고 결측치 채우기 
df_merged = pd.merge(df_merge, df_merge_date, how='outer', on='일시')
print(df_merged[df_merged['기온(°C)'].isnull()]) #결측치 확인 8391개 
df_merged = df_merged.sort_values('일시').reset_index(drop=True)  

                      일시  기온(°C)
263361  1988-01-01 01:00     NaN
263362  1988-01-01 02:00     NaN
263363  1988-01-01 04:00     NaN
263364  1988-01-01 05:00     NaN
263365  1988-01-02 01:00     NaN
...                  ...     ...
271747  2017-09-26 17:00     NaN
271748  2017-10-12 15:00     NaN
271749  2017-10-12 16:00     NaN
271750  2017-10-12 17:00     NaN
271751  2018-11-29 18:00     NaN

[8391 rows x 2 columns]


In [26]:
df_merged = df_merged.interpolate() # 보간하기 

In [27]:
lunar_date = pd.read_csv('./make_data/0B_1988_음력날짜.csv')  ### 2019년 1/1~3/31일까지는 음력으로 11/26~2/25  

In [28]:
lunar_date

,date,Year,Month,Day,lunYear,lunMonth,lunDay
0,1988-10-01,1988,10,1,1988,8,21
1,1988-10-02,1988,10,2,1988,8,22
2,1988-10-03,1988,10,3,1988,8,23
3,1988-10-04,1988,10,4,1988,8,24
4,1988-10-05,1988,10,5,1988,8,25
...,...,...,...,...,...,...,...
11134,2019-03-27,2019,3,27,2019,2,21
11135,2019-03-28,2019,3,28,2019,2,22
11136,2019-03-29,2019,3,29,2019,2,23
11137,2019-03-30,2019,3,30,2019,2,24


In [29]:
df_merged['date']= df_merged['일시'].str[:10] #10년 데이터에서 date날짜 만들기 
df_merged_lunar = pd.merge(df_merged,lunar_date, how='inner', on='date') # 10년 기온데이터와 음력날짜 병합 
df_merged_lunar = df_merged_lunar.sort_values('일시').reset_index(drop=True)
df_merged_lunar['time'] = df_merged_lunar['일시'].str[11:13].astype(int)

In [30]:
df_merged_lunar

,일시,기온(°C),date,Year,Month,Day,lunYear,lunMonth,lunDay,time
0,1988-10-01 00:00,16.4,1988-10-01,1988,10,1,1988,8,21,0
1,1988-10-01 01:00,15.7,1988-10-01,1988,10,1,1988,8,21,1
2,1988-10-01 02:00,15.0,1988-10-01,1988,10,1,1988,8,21,2
3,1988-10-01 03:00,14.3,1988-10-01,1988,10,1,1988,8,21,3
4,1988-10-01 04:00,14.0,1988-10-01,1988,10,1,1988,8,21,4
...,...,...,...,...,...,...,...,...,...,...
265171,2018-12-31 19:00,-3.1,2018-12-31,2018,12,31,2018,11,25,19
265172,2018-12-31 20:00,-3.7,2018-12-31,2018,12,31,2018,11,25,20
265173,2018-12-31 21:00,-4.6,2018-12-31,2018,12,31,2018,11,25,21
265174,2018-12-31 22:00,-5.4,2018-12-31,2018,12,31,2018,11,25,22


In [31]:
df_merged_lunar['date'] = pd.to_datetime(df_merged_lunar['date'], format='%Y-%m-%d')
df_merged_lunar['time'] = df_merged_lunar['time']+1
df_merged_lunar['temp'] = df_merged_lunar['기온(°C)']
df_merged_lunar['year'] = df_merged_lunar['Year']
df_merged_lunar['month'] = df_merged_lunar['Month']
df_merged_lunar['day'] = df_merged_lunar['Day']
df_merged_lunar['weekday'] = df_merged_lunar['date'].dt.weekday    

columns = ['일시','date','time','temp','year','month','day','weekday','lunYear','lunMonth','lunDay']

df_merged_lunar = df_merged_lunar[columns]

In [32]:
df_merged_lunar

,일시,date,time,temp,year,month,day,weekday,lunYear,lunMonth,lunDay
0,1988-10-01 00:00,1988-10-01,1,16.4,1988,10,1,5,1988,8,21
1,1988-10-01 01:00,1988-10-01,2,15.7,1988,10,1,5,1988,8,21
2,1988-10-01 02:00,1988-10-01,3,15.0,1988,10,1,5,1988,8,21
3,1988-10-01 03:00,1988-10-01,4,14.3,1988,10,1,5,1988,8,21
4,1988-10-01 04:00,1988-10-01,5,14.0,1988,10,1,5,1988,8,21
...,...,...,...,...,...,...,...,...,...,...,...
265171,2018-12-31 19:00,2018-12-31,20,-3.1,2018,12,31,0,2018,11,25
265172,2018-12-31 20:00,2018-12-31,21,-3.7,2018,12,31,0,2018,11,25
265173,2018-12-31 21:00,2018-12-31,22,-4.6,2018,12,31,0,2018,11,25
265174,2018-12-31 22:00,2018-12-31,23,-5.4,2018,12,31,0,2018,11,25


In [33]:
df_merged_lunar.to_csv('./make_data/0D_temp_1988_서울.csv', index=False)